In [323]:
# Libraries

import pandas as pd

In [324]:
file = input('Introduce the path file: ').strip('"')

In [325]:
def summaries(oil_type):

    oil_types = {
            'soybean_2025' : {'NA SBO 2025': {'cols': 'd:k', 'skiprows': 3, 'nrows':13}},
            'soybean_2026': {'NA SBO 2026': {'cols': "d:k", 'skiprows': 3,'nrows':13}},
            'soybean_2027':{'NA SBO 2027': {'cols': "d:k", 'skiprows': 3,'nrows':13}},
            'palm_2027':{'NA Palm 2027': {'cols': "e:u", 'skiprows': 16,'nrows':5}},
            'palm_2026': {'NA Palm 2026': {'cols': "e:u", 'skiprows': 16,'nrows':5}},
            'palm_2025':{'NA Palm 2025': {'cols': "e:u", 'skiprows': 16,'nrows':5}},
             }
    
    if 'soybean'in oil_type:    
        for sheet_name, parameters in oil_types[oil_type].items():
            usecols = parameters['cols']
            skiprows = parameters['skiprows']
            nrows = parameters['nrows']

            df = pd.read_excel(
                file,
                sheet_name=sheet_name,
                usecols= usecols,
                skiprows=skiprows,
                nrows=nrows
                )
        
        month_dict = {'Jan':'january', 'Feb':'february', 'Mar':'march', 'Apr':'april', 'May':'may', 'Jun':'june',
                'Jul':'july', 'Aug':'august', 'Sep':'september','Oct':'october', 'Nov':'november', 'Dec':'december'
                }
        cols_to_correct = ['Delivery','Contract']
        for cols in cols_to_correct:
            df[cols] = (df[cols]
                        .astype(str)
                        .str.strip()
                        .replace(month_dict)
                        )
            
        month_to_qtr = {
            'january':'q1', 'february':'q1', 'march':'q1',
            'april':'q2', 'may':'q2', 'june':'q2', 'july':'q3',
            'august':'q3', 'september':'q3', 'october':'q4', 
            'november':'q4', 'december':'q4','FY':'fy'
        }

        df['Qtr'] = df['Delivery'].map(month_to_qtr)
        qtr_to_reorder = df.pop('Qtr')
        df.insert(2,'Qtr',qtr_to_reorder)
        
        df['spend'] = df['Purchased']*df['CVG FUT']
        df_summary = df.groupby('Qtr').agg(
        cov_volume=('Purchased','sum'),
        spend=('spend','sum'))

        df_summary['cov_rate'] = df_summary['spend']/df_summary['cov_volume']
        df_summary = df_summary.drop(columns=['spend'])
        df_summary = df_summary.T

    else:
            
            for sheet_name, parameters in oil_types[oil_type].items():
                usecols = parameters['cols']
                skiprows = parameters['skiprows']
                nrows = parameters['nrows']

                df = pd.read_excel(
                    file,
                    sheet_name=sheet_name,
                    usecols= usecols,
                    skiprows=skiprows,
                    nrows=nrows
                    )
            
            df_summary = df.rename(columns={'Unnamed: 4':'Qtr'})

            palm_filtered = ['Qtr','Volume (lbs)','Purchased (lbs)','% Coverage (Base Commodity)','CVG FUT (₵/lb)']
            df_summary= df_summary[palm_filtered]

    return df_summary
    

In [ ]:
print('-'*73)
print('Soybean 2025:')
print(summaries('soybean_2025').to_string())
print('-'*73)
print('Soybean 2026:')
print(summaries('soybean_2026').to_string())
print('-'*73)
print('Soybean 2027:')
print(summaries('soybean_2027').to_string())
print('-'*82)
print('Palm 2025:')
print(summaries('palm_2025').to_string())
print('-'*82)
print('Palm 2026:')
print(summaries('palm_2026').to_string())
print('-'*82)
print('Palm 2027:')
print('-'*82)